In [ ]:
# conda install ipyparallal

# ipcluster nbextension enable

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from IPython.core.pylabtools import figsize
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
figsize(11, 9)

for pd_option in ['display.max_rows', 'display.max_colwidth', 'display.max_columns']:
    pd.set_option(pd_option, 500)  # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

In [2]:
import paramiko 

class DevboxCMD:
    """
    Base class for connecting to the MDM DevBox and executing commands.
    Possible commands include everything from querying data, executing unix commands, to copying files
    """
    def __init__(self):
        ssh_loc = 'C:\\Users\\location_name\\private_key_openssh.key'
        self.myPkey = paramiko.RSAKey.from_private_key_file(ssh_loc)
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.client.connect(hostname='', username='', pkey=self.myPkey)
        self.cwd = self.client.exec_command('')

    def mdm_cmd(self, command):
        self.stdin, self.stdout, self.stderr = self.client.exec_command(command)

In [3]:
from sshtunnel import SSHTunnelForwarder #Run pip install sshtunnel
import paramiko

def get_mysql_connection():
    ssh_loc = 'C:\\Users\\location_name\\private_key_openssh.key'
    myPkey = paramiko.RSAKey.from_private_key_file(ssh_loc)
    server = SSHTunnelForwarder(
        ssh_address_or_host=('', 22), 
        ssh_username='', 
        ssh_pkey=myPkey, 
        # local_bind_address=('127.0.0.1', 5999),
        remote_bind_address=('127.0.0.1', 3306) )
    server.start()
    
    local_port = str(server.local_bind_port)
    engine = create_engine('mysql+pymysql://root:@localhost:' + local_port +'/insightsquared')
    return server, engine


In [4]:
engine = create_engine('sqlite:///fake_db_test.db')
connection_to_db = str(engine.url)

In [8]:
%load_ext sql
%config SqlMagic.displaylimit = 5
%sql $connection_to_db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: None@fake_db_test.db'

In [1]:
from ipyparallel import Client

cluster_clients = Client()
cluster_clients.block = True  # use synchronous computations
print "\nThere are {} clients with the following id's {}\n".format(len(cluster_clients.ids), cluster_clients.ids)
view = cluster_clients.load_balanced_view()
view.block = True


There are 4 clients with the following id's [0, 1, 2, 3]



In [3]:
all_engines = cluster_clients[:]
all_engines

<DirectView [0, 1, 2, 3]>

In [22]:
def my_sum(a, b):
    
    return a + b

my_sum_apply_results = all_engines.apply(my_sum, 11, 31)
my_sum_apply_results

[42, 42, 42, 42]

In [ ]:
my_sum_apply_results.get()

In [4]:
with all_engines.sync_imports():
    import numpy
    import pandas
    from sqlalchemy import create_engine
    import paramiko

importing numpy on engine(s)
importing pandas on engine(s)
importing create_engine from sqlalchemy on engine(s)
importing paramiko on engine(s)


In [13]:
pd.__version__

u'0.18.1'

In [26]:
def slow_var_test(a):
    return pandas.__version__, x, my_sum(1, 2)

In [27]:
results = view.map(slow_var_test, [0, 1, 2, 3])

In [28]:
results

[(u'0.18.1', 'f', 3),
 (u'0.18.1', 'f', 3),
 (u'0.18.1', 'f', 3),
 (u'0.18.1', 'f', 3)]

In [23]:
all_engines.push(dict(x='f', my_sum=my_sum))

[None, None, None, None]